1. generate small cutouts
2. measure r_cut from the (or multiple, his) protonatable sites
3. measure r_crit "" ""
4. SAVE PDB
4. check for protonatable sites from neighboring residues. NO? x{d,p}
Yes? 1 =  x{}, triangle{}
1+?  [x{},triangle{}]

MAX TWO AT A TIME, NO "TAKING"

5: COMBINE selected residues cutout radius and delete overlap


todo: fix that NE2 goes to gln.
-navigate when theres 1+ protonation site in cutout saving schemes


In [1]:
from Bio.PDB import *
from Bio import PDB
from torch import nn
import numpy as np
from ase import Atoms, Atom
import dask.dataframe as dd
from ordered_set import OrderedSet
import os

local_folder="/Users/jessihoernschemeyer/pKaSchNet"
pkPDB_CSV = f"{local_folder}/pkas.csv"
def read_database(path):
    """csv --> dask df"""
    #make the dask data frame from the PYPKA csv
    dk=dd.read_csv(pkPDB_CSV, delimiter=';', na_filter=False, dtype={'idcode':'category', 
                                                                    'residue_number':'uint8',
                                                                    'pk': 'float32',
                                                                    'residue_name':'category',
                                                                    'chain': 'category',
                                                                    'residue_name': 'category'
                                                                    })
                                                            
    dk=dk.rename(columns={'idcode': 'PDB ID', 'residue_number': 'Res ID', 'residue_name': 'Res Name', 'residue_number': 'Res ID', 'pk': 'pKa', 'chain' : 'Chain'}) #rename columns to match df from pkad 
    dk=dk.sort_values(['PDB ID', 'Res ID'], ascending=[True, True]) 
    dk=dk.compute() 
    dff = dk.reset_index() 

    return dff

def check_atoms_protein(structure, struc_atoms): 
    """internal function. checks every atom in the entire protein for metals, undesirables"""
    pdb_residues=[]
    for atom in struc_atoms: 
        resname, atomid=atom.get_parent().get_resname(), atom.get_full_id()[2:]
        element=atomid[2][0]

        if element in ["MG", "MN", "FE", "CO", "NI", "CU", "ZN"]:
            return 0,0#print(f"{element} present, pdb skipped")
        
        else:
            #atomid=atom.get_full_id() #('', 0, 'B', (' ', 177, ' '), ('OH', ' '))
            if atomid[1][0] not in [' ']:
                if element == 'S': #check 4 hetero sulfur, exclude.
                    print(f"{atomid}, hetero sulfur. pdb skipped ")
                    return 0,0
                
                if element in ['CA', 'CL', 'K', 'NA']: #other salt
                    for res in structure.get_residues():
                        if resname in ["GLU", "HIS", "ASP", "ARG", "TYR", "CYS", "LYS"]: #if the other salt is part of the residue (<3å from geometric center), delete atom from residue
                            if np.linalg.norm(res.center_of_mass(geometric=True) - atom.get_coord()) < 3:
                                atom.get_parent().detach_child(atom.get_id()) #print(f"salt {atom} deleted, {d} from {res}")

        pdb_residues.append(atomid[0] + str(atomid[1][1]))
    
    return structure, set(pdb_residues) #('', 0, 'B', ('W', 371, ' '), ('O', ' '))

def atoms_to_structure(cutout, filename): 
    """Internal function (or not), cutout --> save to harddrive
    input: cutout: list of biopython atom objects (NOT ASE)"""
    chain_dict = {}

    structure = Structure.Structure(filename)
    model = Model.Model(0)
    structure.add(model)

    for atom in cutout:
        res = atom.get_parent()
        res_id, resname, chain_id = res.get_id(), res.get_resname(), res.get_full_id()[2]

        if chain_id not in chain_dict:
            chain = Chain.Chain(chain_id) #make new chain
            chain_dict[chain_id] = chain
            model.add(chain) #add it

        else:
            chain = chain_dict[chain_id]

        if res_id in [r.get_id() for r in chain.get_residues()]:
            residue = [r for r in chain.get_residues() if r.get_id() == res_id][0]
        else:
            residue = Residue.Residue(res_id, resname, '') #make res
            chain.add(residue)

        residue.add(atom)
    # save the pdb
    io = PDBIO()
    io.set_structure(structure)
    io.save(f"{filename}.pdb")

dask_df = read_database(local_folder + pkPDB_CSV)

The index of cuts matches the index .. no it doesnt. to match the cut index to the db will need a dictionary, like his-12-E = 100th pypka point = 103rd cutout = HID-13



In [4]:
pdb_parser, pdbs = PDB.PDBParser(), list(OrderedSet(list(dask_df["PDB ID"])))
def generate_cutout_around_protonatable_site(residue, distance_cutoff, ns, counter, fname):
    """Residue wise resolurion. ns is neighbor search set up for the entire protein, residue is the single data point / 1 of several residues in a pdb & in pypka.
    input is one residue. output is the cutout around its titratable site, both of which can be plural e.g. his, mb asp and glu.
    residue (biopython Residue object): a single protonable residue """
    protonatable_sites = ["OE2", "SG", "NZ", "OD2", "NE2", "ND1", "OH"]
    cuts = []
    for atom in residue.get_atoms():
        if atom.is_disordered():
            atomN = str(atom)[16:-1]
            if atomN in protonatable_sites:
                return 
        atomN = str(atom)[6:-1]#, residue.get_resname()[0]
        
        if atomN == protonatable_sites[0]: #glu
            center = atom.get_coord()
            cut = ns.search(center, distance_cutoff, "A")
            cuts.append((counter, center, residue.get_resname()[0], cut)) #counter is id!
            
            continue
        if atomN  == protonatable_sites[1]: #CYS
            center = atom.get_coord()
            cut = ns.search(center, distance_cutoff, "A")
            cuts.append((counter, center, residue.get_resname()[0], cut))
            return cuts
        if atomN == protonatable_sites[2]: #LYS
            center = atom.get_coord()
            cut = ns.search(center, distance_cutoff, "A")
            cuts.append((counter, center, residue.get_resname()[0], cut))
            return cuts
        if atomN ==protonatable_sites[3]: #ASP
            center = atom.get_coord()
            cut = ns.search(center, distance_cutoff, "A")
            cuts.append((counter, center, residue.get_resname()[0], cut))
            continue
########HIS 
        if atomN == protonatable_sites[4]: #his eps #TODO: maybe one always comes first in biopython?
            center = atom.get_coord()
            cut = ns.search(center, distance_cutoff, "A")
            cuts.append((str(counter) + 'E', center, residue.get_resname()[0], cut))
            continue
        if atomN == protonatable_sites[5]: #HIS #TODO
            center = atom.get_coord()
            cut = ns.search(center, distance_cutoff, "A")
            cuts.append((str(counter) + 'D', center, residue.get_resname()[0], cut))
            print(cuts)
            continue

        if atomN == protonatable_sites[6]: #TYR
            center = atom.get_coord()
            cut = ns.search(center, 6, "A")
            cuts.append((counter, center, residue.get_resname()[0], cut))
            return cuts

    return cuts #plural because of sites with multiple sites.


In [8]:
def merge_or_not_cutouts(cutouts_apdb, distance_cutoff): #TODO: reduce dtypes #PDB WISE!
    """
    in: all of the cutouts from the pdb. returns the merged or solo cutout for each input residue of cutouts_apdb. len in = len out"""
    #protein wise ..
    ids,centers,cuts, Ds_lite, counter, cutouts, cuts_dict =[],[],[],[], 0, [], {}
    for site in cutouts_apdb:
        ids.append(site[0])
        centers.append(site[1])
        cuts.append(site[3])
    
    num_residues=len(centers)
    distances = np.zeros((num_residues, num_residues))

    for i in range(num_residues): #residue wise..
        for j in range(i + 1, num_residues):
            distance = np.linalg.norm(centers[i] - centers[j]).astype(np.float32)
            if distance < distance_cutoff:
                distances[i, j] = distance
                distances[j, i] = distance

    for i in range(len(distances)):
        Ds_lite.append(np.array([distances[:,i][j] for j in range(num_residues) if distances[:,i][j] != 0])) #goes column wise and just gets the nonzeros

#now residue wise.... 
    for a_residues_distance_array in Ds_lite: #ds lite is nontrivial distances
        if a_residues_distance_array.size != 0: #if not empty
            closest_cutout_i = int(np.where((distances[:,counter])==np.min(a_residues_distance_array))[0])
            pair_i = frozenset((counter,closest_cutout_i)) #key
            if not bool(cuts_dict.keys()): #if there are not any pairs yet generated, make some..

                if pair_i not in cuts_dict.keys():
                    closest_cutout = cuts[closest_cutout_i]
                    cutout = list(set(cuts[counter] + closest_cutout))
                    cuts_dict[pair_i] = cutout
                else:
                    cutout = cuts_dict[pair_i] #"cutout = dict entry
                    
            counter += 1

        else: #solo cutout
            cutout = cuts[counter]
            counter += 1
        cutouts.append(cutout)
    return cutouts

def get_cutout(dask_df=dask_df, distance_cutoff=6): #"PARENT" FUNCTION
    """for each protein in dask_df (the entire PYPKA database), it iterates residue wise through the 121,294 proteins in PYPKA database and downloads
    the structure from RCSB with biopython. Then, it checks and skips the structure if metals & hetero sulfurs are present, and deletes non-sulfur
    salts from titratable residues.
    Then, for each structure residue represented in PYPKA, generates a cutout for each residue, appends the structure to cutouts_apdb"""
    pdbname="11as"  #for now
    cutouts_apdb, all_tit_res =[],[]
    for i in range(18,19): #will equal len of set of pdbs in pypka, == 121294 
        cutouts_apdb, fnames, cutouts_1_datapoint, counter, pdbname =[],[], [],0, pdbs[i]
        Structure = pdb_parser.get_structure("",  PDBList().retrieve_pdb_file(str.lower(pdbname),obsolete=False, pdir='PDB',file_format = 'pdb'))
        structure, pdb_residues = check_atoms_protein(Structure, Structure.get_atoms())
        if structure==0: #skip entire pdb and all its entries in pypka db if there are undesirables in pdb
            continue

        ns = PDB.NeighborSearch(list(structure.get_atoms())) #set up ns , entire protein
        pdb_df = dask_df[dask_df.iloc[:, 1] == pdbname].drop(columns = ["PDB ID", "pKa"]) #make a subdf containing only residue entries which are in PYPKA (dask_df) 
        
        for j in range(len(pdb_df)):  #go through each residue in a pdb #each j is a datapoint!
            chain, res_id =pdb_df.iloc[j]['Chain'], int(pdb_df.iloc[j]['Res ID'])
            
            try: 
                residue=structure[0][chain][res_id] #a datapoint #TODO: make ID?
                pypka_resname, PDBresname = pdb_df.iloc[j]['Res Name'], residue.get_resname() #pypka error
                if pypka_resname not in [PDBresname]: #ACHTUNG! this navigates the pyka error. #TODO: mail him #TODO: THIS EXCLUDES NTR AND CTR!
                    continue

                #generate the cutout solo cutouts
                cutouts_1_datapoint=generate_cutout_around_protonatable_site(residue, distance_cutoff, ns, counter,  f"{pdbname}_{chain}_{res_id}_{PDBresname}") #can be multiple
                #returns empty if disordered
                
                cutouts_apdb.append(*cutouts_1_datapoint) #append each residue/data point cutouts here #it will error here if disordered
                fnames.append(f"{pdbname}_{chain}_{res_id}_{PDBresname}-{cutouts_1_datapoint[0][0]}") 

                all_tit_res.append(chain + str(res_id))
                #print(f"{pdbname}_{res_id}_{chain}_{PDBresname[0]}-{cutouts_1_datapoint[0][0]}")
                counter+=1 #counter doesnt reach here if it fails

            except: 
                #print(f"{pdbname}_{res_id}_{chain}_{PDBresname}")
                print("except")
                pass #means pypka res not found in PDB

        #os.remove(f"{local_folder}/PDB/pdb{pdbname}.ent")  
        if cutouts_apdb:
            merged_and_solos=merge_or_not_cutouts(cutouts_apdb, distance_cutoff)#make a merged cutout or not based off radius criteria
            for cut, fname in zip(merged_and_solos, fnames):
                atoms_to_structure(cut, fname) #save as pdb)
            lines = str(all_tit_res)[1:-1].strip()
            with open(f"{pdbname}", 'w') as file: #save all the residue names 
                file.write(lines)

    return 
#os.remove(f"{local_folder}/PDB/pdb{pdbname}.ent")"""

get_cutout(dask_df=dask_df, distance_cutoff=6)

Structure exists: 'PDB/pdb194l.ent' 
except
[('2D', array([-9.412, 12.961, 16.139], dtype=float32), 'H', [<Atom O>, <Atom N>, <Atom CG>, <Atom CD>, <Atom CA>, <Atom CB>, <Atom N>, <Atom NH1>, <Atom NE>, <Atom CZ>, <Atom NH2>, <Atom C>, <Atom O>, <Atom O>, <Atom C>, <Atom CA>, <Atom CA>, <Atom N>, <Atom N>, <Atom O>, <Atom CB>, <Atom O>, <Atom CD1>, <Atom CG1>, <Atom O>, <Atom C>, <Atom N>, <Atom C>, <Atom CA>, <Atom CB>, <Atom CG>, <Atom CD2>, <Atom ND1>, <Atom NE2>, <Atom CE1>, <Atom OG1>, <Atom CG2>, <Atom OD1>, <Atom O>, <Atom CB>, <Atom CB>, <Atom CA>, <Atom N>, <Atom O>, <Atom CG1>, <Atom N>, <Atom C>, <Atom O>, <Atom CB>, <Atom N>, <Atom CA>])]
except


/var/folders/jl/8mbcps0x1jv9616svy8_9vy80000gn/T/ipykernel_38102/1570701517.py:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  closest_cutout_i = int(np.where((distances[:,counter])==np.min(a_residues_distance_array))[0])


residue 1,2,3 =/= residue number 1, 2 3!!! correspond with successful residues from the pypka. TODO: really should make a dict index pdb to #.

things that are needed...
1. protonate/change name only if the other residue is ... 

In [236]:
#working to combine the merging or not and distance stuffimport numpy as np
#HEREE
#coords = [c[1] for c in cutouts_apdb[0][0]]#replace 0 with an iterative. each slice is a pdb
#cuts = [c[3] for c in cutouts_apdb[0][0]]

def merge_or_not_cutouts(cutouts_apdb, distance_cutoff): #TODO: reduce dtypes #PDB WISE!
    """
    in: all of the cutouts from the pdb. returns the merged or solo cutout for each input residue of cutouts_apdb. len in = len out"""
    #print(cutouts_apdb)
    #protein wise ..
    ids,centers,cuts=[],[],[]
    #print(*cutouts_apdb)
    for site in cutouts_apdb:
        print(3, site)
        #print(pypka_datapoint)
        #for Site in pypka_datapoint:
        #print(pypka_datapoint)
            #for site in Site:
                #print("HELLO", site[0])
        ids.append(site[0])
            #print(site, "HI")
        centers.append(site[1])
        cuts.append(site[3])
    print(4,ids)
    num_residues=len(centers)
    distances = np.zeros((num_residues, num_residues))

    for i in range(num_residues): #residue wise..
        for j in range(i + 1, num_residues):
            distance = np.linalg.norm(centers[i] - centers[j]).astype(np.float32)
            if distance < distance_cutoff:
                distances[i, j] = distance
                distances[j, i] = distance
    #print(dist)
    Ds_lite=[]
    for i in range(len(distances)):
        Ds_lite.append(np.array([distances[:,i][j] for j in range(num_residues) if distances[:,i][j] != 0])) #goes column wise and just gets the nonzeros
    print(Ds_lite)
#now residue wise.... 
    counter, cutouts, cuts_dict=0, [], {}

    for a_residues_distance_array in Ds_lite: #ds lite is nontrivial distances
    
        if a_residues_distance_array.size != 0: #if not empty
            closest_cutout_i = int(np.where((distances[:,counter])==np.min(a_residues_distance_array))[0])
            pair_i = frozenset((counter,closest_cutout_i)) #key
            print(pair_i)
            if not bool(cuts_dict.keys()):

                if pair_i not in cuts_dict.keys():
                    print(3)
                    closest_cutout = cuts[closest_cutout_i]
                    
                    cutout = list(set(cuts[counter] + closest_cutout))
                    print("m", cutout)
                    cuts_dict[pair_i] = cutout
                else:
                    cutout = cuts_dict[pair_i]
                    print("cutout = dict entry")
            counter += 1

        else:
            cutout = cuts[counter]
            print("solo cutout makes boolean false")
            counter += 1
        cutouts.append(cutout)
    #print("cutouts", cutouts)
    return cutouts

merged_hoodies = merge_or_not_cutouts(cutouts_apdb, 6) #working ##
print(set(merged_hoodies[0]) == set(merged_hoodies[1]), ":)")

3 (0, array([20.947, 40.838, 23.05 ], dtype=float32), 'T', [<Atom OXT>, <Atom NE2>, <Atom CD>, <Atom OE1>, <Atom O>, <Atom CG>, <Atom CB>, <Atom NE2>, <Atom CD2>, <Atom CE1>, <Atom CG>, <Atom CD1>, <Atom O>, <Atom OG1>, <Atom CZ2>, <Atom CA>, <Atom CB>, <Atom CG2>, <Atom C>, <Atom OH>, <Atom O>, <Atom CZ>, <Atom CB>, <Atom CG>, <Atom CD2>, <Atom CE2>, <Atom CG2>, <Atom CE1>, <Atom CZ>, <Atom C>, <Atom N>, <Atom C>, <Atom CA>, <Atom N>, <Atom CH2>, <Atom CZ3>, <Atom O>, <Atom CG1>, <Atom CB>, <Atom CD1>, <Atom CG>, <Atom CB>])
3 (1, array([43.749, 21.409, 17.445], dtype=float32), 'T', [<Atom CE2>, <Atom OG1>, <Atom CA>, <Atom CB>, <Atom CD1>, <Atom CZ3>, <Atom C>, <Atom CA>, <Atom CH2>, <Atom O>, <Atom CG1>, <Atom CZ2>, <Atom N>, <Atom CB>, <Atom C>, <Atom CG2>, <Atom CA>, <Atom OE1>, <Atom C>, <Atom CE1>, <Atom O>, <Atom CG2>, <Atom CD2>, <Atom C>, <Atom O>, <Atom OXT>, <Atom OH>, <Atom CE1>, <Atom CZ>, <Atom CD1>, <Atom CE2>, <Atom CG>, <Atom CD2>, <Atom NE2>, <Atom CD>, <Atom CG>, <A

#cutouts_apdb[PDB][RESIDUE1,2,3..][which protonation state e.g. his, gonna need for __ site in resdatapoint here..]
cutouts_apdb[0][0][0]
#a_single_cutout[cutout id, was dt by counter which should correspond with the values on pdb_df as well as the second slice "residue". ]

"""pdb_parser, pdbs = PDB.PDBParser(), list(OrderedSet(list(dask_df["PDB ID"])))

def generate_cutout_around_protonatable_site(residue, distance_cutoff, ns, counter):
    """ns is neighbor search set up for the entire protein, residue is the single data point / 1 of several residues in a pdb & in pypka.
    input is one residue. output is the cutout around its titratable site, both of which can be plural e.g. his, mb asp and glu."""
    cuts = []
    print(3)
    for atom in residue.get_atoms():
        atomN = str(atom)[6:-1]#, residue.get_resname()[0]
        if atomN == "OE2": #glu
            center = atom.get_coord()
            cut = ns.search(center, distance_cutoff, "A")
            cuts.append((counter, center, residue.get_resname()[0], cut))
            
            continue
        if atomN  == 'SG': #CYS
            center = atom.get_coord()
            cut = ns.search(center, distance_cutoff, "A")
            cuts.append((counter, center, residue.get_resname()[0], cut))
            return cuts
        if atomN =="NZ": #LYS
            center = atom.get_coord()
            cut = ns.search(center, distance_cutoff, "A")
            cuts.append((counter, center, residue.get_resname()[0], cut))
            return cuts
        if atomN =="OD2": #ASP
            center = atom.get_coord()
            cut = ns.search(center, distance_cutoff, "A")
            cuts.append((counter, center, residue.get_resname()[0], cut))
            continue
########HIS 
        if atomN == "NE2": #his eps #TODO: maybe one always comes first in biopython?
            center = atom.get_coord()
            cut = ns.search(center, distance_cutoff, "A")
            cuts.append((str(counter) + 'E', center, residue.get_resname()[0], cut))
            continue
        if atomN == "ND1": #HIS #TODO
            center = atom.get_coord()
            cut = ns.search(center, distance_cutoff, "A")
            cuts.append((str(counter) + 'D', center, residue.get_resname()[0], cut))
            continue

        if atomN == "OH": #TYR
            center = atom.get_coord()
            cut = ns.search(center, 6, "A")
            cuts.append((counter, center, residue.get_resname()[0], cut))
            return cuts

    return cuts #plural because of sites with multiple sites."""

In [ ]:
import numpy as np

#coords = [c[1] for c in cutouts_apdb[0][0]]#replace 0 with an iterative. each slice is a pdb
#cuts = [c[3] for c in cutouts_apdb[0][0]]

def compute_interatomic_distances(cutouts_apdb, distance_cutoff): #TODO: reduce dtypes
    """
    in: all of the cutouts from the pdb

    Compute an array of interatomic distances for a given set of atomic coordinates.

    Parameters:
    coords (numpy.ndarray): An array of shape (N, 3) representing the 3D coordinates of N atoms.

    Returns:
    numpy.ndarray: A 2D array of shape (N, N) where element (i, j) is the distance between atom i and atom j.
    """
    coords, cuts = [c[0][1] for c in cutouts_apdb], [c[0][3] for c in cutouts_apdb]
    num_residues=len(coords)
    distances = np.zeros((num_residues, num_residues))
    

    for i in range(num_residues):
        for j in range(i + 1, num_residues):
            distance = np.linalg.norm(coords[i] - coords[j])
            if distance < distance_cutoff:
                distances[i, j] = distance
                distances[j, i] = distance

    Ds=[]
    for i in range(len(distances)):
    #remove diag zeros
        #Ds.append([distances[:,i][j] for j in range(len(a)) if distances[:,i][j] != 0])
        Ds.append([distances[:,i][j] for j in range(num_residues)])
    return Ds #for each cutout in pdbs_acutout, this is a list 

distances = compute_interatomic_distances(cutouts_apdb, 6)
distances

In [150]:
#working to combine the merging or not and distance stuffimport numpy as np

#coords = [c[1] for c in cutouts_apdb[0][0]]#replace 0 with an iterative. each slice is a pdb
#cuts = [c[3] for c in cutouts_apdb[0][0]]

def compute_interatomic_distances(cutouts_apdb, distance_cutoff): #TODO: reduce dtypes
    """
    in: all of the cutouts from the pdb

    Compute an array of interatomic distances for a given set of atomic coordinates.

    Parameters:
    coords (numpy.ndarray): An array of shape (N, 3) representing the 3D coordinates of N atoms.

    Returns:
    numpy.ndarray: A 2D array of shape (N, N) where element (i, j) is the distance between atom i and atom j.
    """
    #protein wise ..
    coords, cuts = [c[0][1] for c in cutouts_apdb], [c[0][3] for c in cutouts_apdb]
    num_residues=len(coords)
    distances = np.zeros((num_residues, num_residues))

    for i in range(num_residues): #residue wise..
        for j in range(i + 1, num_residues):
            distance = np.linalg.norm(coords[i] - coords[j])
            if distance < distance_cutoff:
                distances[i, j] = distance
                distances[j, i] = distance

    Ds_lite=[]
    #print(distances)
    for i in range(len(distances)):
        Ds_lite.append(np.array([distances[:,i][j] for j in range(num_residues) if distances[:,i][j] != 0])) #goes column wise and just gets the nonzeros
        #for j in range(len(distances)):
            #if distances[:,i][j] != 0:
                #Ds.append(distances[:,i][j])

                #print(distances[:,i])
            #[distances[:,i][j] for j in range(num_residues) if distances[:,i][j] != 0])
    #remove diag zeros
        #print(([distances[:,i][j] for j in range(num_residues) if distances[:,i][j] != 0]))
    #print(np.array([distances[:,i][j] for i,j in range(num_residues), range(num_residues) if distances[:,i][j] != 0])) #keep only the numbers
    #print(Ds)
#now residue wise.... 
    #a_residues_distance_array, input_=Ds[0], cuts[0]
    #print(Ds)
    counter=0
    cutouts=[]
    for a_residues_distance_array in Ds_lite:
        
        if a_residues_distance_array:
            print(a_residues_distance_array)
            minn = np.min(a_residues_distance_array)
            
            closest_cutout_i = np.where((distances[:,counter])==minn)
            print(closest_cutout_i)
            #closest_cutout = cuts[np.where(np.array(distances[0])==minn)[0]]
                #cutout = input_cut + closest_cutout
                #print("cutout = input cutout A + the base cutout with index minn") #in the tuple u
            #cutout = set(cuts[counter] + closest_cutout)
        #for d in a_residues_distance_array: #for a residue's distances in all cutouts center's distances to each other 
            #print(d)
            #minn = np.min(d) #minimum of the distances
            counter += 1
        else:
            cutout = cuts[counter]
            counter += 1
            #else:
                #closest_cutout_i = np.where(np.array(distances[0])==minn)
                #closest_cutout = cuts[closest_cutout_i]
                #cutout = input_cut + closest_cutout
                #print("cutout = input cutout A + the base cutout with index minn") #in the tuple u
            #cutout = set(input_cut + closest_cutout)
        cutouts.append(cutout)
    return cutout
#coords = np.array([3*c[0][1] for c in cutouts_apdb[0]] + [c[0][1] for c in cutouts_apdb[0]])
    

    return Ds #for each cutout in pdbs_acutout, this is a list 

Ds = compute_interatomic_distances(cutouts_apdb, 6)

        
a = merge_with_closest_residue_or_not(a_residues_distance_array, cuts)

/var/folders/jl/8mbcps0x1jv9616svy8_9vy80000gn/T/ipykernel_30174/2624738790.py:51: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if a_residues_distance_array:


[3.74191499]
(array([60]),)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
rcut = 113

#for a cutout A..
#minn = np.min(Ds)
a_residues_distance_array=Ds[0]
def merge_with_closest_residue_or_not(a_residues_distance_array, cuts): 
    input_cut=cuts[0]
    for d in a_residues_distance_array: #for a residue's distances in all cutouts center's distances to each other 
        minn = np.min(d) #minimum of the distances

        if minn != 0:
            cutout = input_cut
        else:
            closest_cutout_i = np.where(np.array(distances[0])==minn)
            closest_cutout = cuts[closest_cutout_i]
            #cutout = input_cut + closest_cutout
            print("cutout = input cutout A + the base cutout with index minn") #in the tuple u
            cutout = set(input_cut + closest_cutout)
        return cutout
#coords = np.array([3*c[0][1] for c in cutouts_apdb[0]] + [c[0][1] for c in cutouts_apdb[0]])

In [114]:
Ds

[[30.47676658630371,
  6.649082183837891,
  32.392601013183594,
  12.277155876159668,
  19.805187225341797,
  24.664180755615234,
  21.784460067749023,
  24.213897705078125,
  13.363728523254395,
  20.113821029663086,
  29.199378967285156,
  21.484466552734375,
  10.017365455627441,
  14.461064338684082,
  33.83466720581055,
  18.468353271484375,
  28.707502365112305,
  30.133607864379883,
  26.728893280029297,
  29.09185218811035,
  31.331098556518555,
  33.10321044921875,
  30.53982925415039,
  30.02446746826172,
  32.290374755859375,
  42.592262268066406,
  26.809768676757812,
  31.514732360839844,
  20.994142532348633,
  31.514732360839844,
  20.994142532348633,
  27.975635528564453,
  21.04035186767578,
  35.34115982055664,
  18.40313720703125,
  28.27549171447754,
  12.135529518127441,
  36.4132080078125,
  6.529940605163574,
  13.015357971191406,
  42.91936492919922,
  13.901250839233398,
  38.369998931884766,
  22.83025550842285,
  12.128716468811035,
  12.33933162689209,
  18.

In [73]:

rcut = 113

#for a cutout A..
#minn = np.min(Ds)

def merge_with_closest_residue_or_not(a_residues_distance_array):
    for d in a_residues_distance_array: #for a residue's distances in all cutouts center's distances to each other 
        minn = np.min(d) #minimum of the distances

        if minn > rcut:
            print("i am the cutout A")
            cutout = input_cut
        else:
            closest_cutout_i = np.where(np.array(distances[0])==minn)
            cutout = input_cut + acutoutB
            print("cutout = input cutout A + the base cutout with index minn") #in the tuple u
            cutout = set(input_cut + acutoutB)
#coords = np.array([3*c[0][1] for c in cutouts_apdb[0]] + [c[0][1] for c in cutouts_apdb[0]])
        
merge_with_closest_residue_or_not(distances[0])


IndexError: list index out of range

In [146]:
"""import numpy as np
for cutouts_aresidue in cutouts_apdb[0]:
    #coord = cutouts_aresidue[1]
    print(cutouts_aresidue)

    
    #if atom.
    ds=[]
    r_c, r_cut = 11,12
    for i in range(len(cutouts_aresidue)):
        print(2)
        for j in range(i + 1, len(cutouts_aresidue)):
            print(3)
            ic, jc = cutouts_aresidue[i][1], cutouts_aresidue[j][1]
            print(ic, jc)
            if ic[0] != jc[0]:
                coord1 = np.array(ic[1:])  # Exclude atom name
                coord2 = np.array(jc[1:]) # Exclude atom name
                print(coord1)
                d=np.linalg.norm(coord2 - coord1)
                #if d < rc:
                    #rcs.append(d)
                
                ds.append(("".join([str(i),str(j)]),(d,coord1,coord2)))"""


[(0, array([20.947, 40.838, 23.05 ], dtype=float32), 'T', [<Atom OXT>, <Atom NE2>, <Atom CD>, <Atom OE1>, <Atom O>, <Atom CG>, <Atom CB>, <Atom NE2>, <Atom CD2>, <Atom CE1>, <Atom CG>, <Atom CD1>, <Atom O>, <Atom OG1>, <Atom CZ2>, <Atom CA>, <Atom CB>, <Atom CG2>, <Atom C>, <Atom OH>, <Atom O>, <Atom CZ>, <Atom CB>, <Atom CG>, <Atom CD2>, <Atom CE2>, <Atom CG2>, <Atom CE1>, <Atom CZ>, <Atom C>, <Atom N>, <Atom C>, <Atom CA>, <Atom N>, <Atom CH2>, <Atom CZ3>, <Atom O>, <Atom CG1>, <Atom CB>, <Atom CD1>, <Atom CG>, <Atom CB>])]
2
[(1, array([43.749, 21.409, 17.445], dtype=float32), 'T', [<Atom CE2>, <Atom OG1>, <Atom CA>, <Atom CB>, <Atom CD1>, <Atom CZ3>, <Atom C>, <Atom CA>, <Atom CH2>, <Atom O>, <Atom CG1>, <Atom CZ2>, <Atom N>, <Atom CB>, <Atom C>, <Atom CG2>, <Atom CA>, <Atom OE1>, <Atom C>, <Atom CE1>, <Atom O>, <Atom CG2>, <Atom CD2>, <Atom C>, <Atom O>, <Atom OXT>, <Atom OH>, <Atom CE1>, <Atom CZ>, <Atom CD1>, <Atom CE2>, <Atom CG>, <Atom CD2>, <Atom NE2>, <Atom CD>, <Atom CG>, <

anyways u need a dictionary to go back to atomwise

working hereeee

In [9]:
#cuts = [c[0][3] for c in cutouts_apdb[0]]
for i in range(len(a)- 1):
    b=[a[:,i][j] for j in range(len(a)) if a[:,i][j] != 0]

Ds=[]
for i in range(len(a)):
    b=[a[:,i][j] for j in range(len(a)) if a[:,i][j] != 0]
    Ds.append(b)




rcut = 113
acutoutB = cuts[1]
#for a cutout A..
#minn = np.min(Ds)
for d in Ds: #for a residue's distances in all cutouts center's distances to each other 
    minn = np.min(d)
    input_cut = cuts[0]
    if minn > rcut:
        print("i am the cutout A")
        cutout = input_cut
    else:
        cutout = input_cut + acutoutB
        print("cutout = input cutout A + the base cutout with index minn") #in the tuple u
        cutout = set(input_cut + acutoutB)
#coords = np.array([3*c[0][1] for c in cutouts_apdb[0]] + [c[0][1] for c in cutouts_apdb[0]])
        
cutout


cutout = input cutout A + the base cutout with index minn
cutout = input cutout A + the base cutout with index minn
cutout = input cutout A + the base cutout with index minn
cutout = input cutout A + the base cutout with index minn


{<Atom O>,
 <Atom CB>,
 <Atom CD1>,
 <Atom CD2>,
 <Atom CE1>,
 <Atom CE2>,
 <Atom CG>,
 <Atom CZ>,
 <Atom OH>,
 <Atom CB>,
 <Atom CD>,
 <Atom CG>,
 <Atom NE2>,
 <Atom OE1>,
 <Atom CA>,
 <Atom C>,
 <Atom CB>,
 <Atom CG1>,
 <Atom CG2>,
 <Atom N>,
 <Atom CA>,
 <Atom C>,
 <Atom O>,
 <Atom CE2>,
 <Atom CH2>,
 <Atom CZ2>,
 <Atom CZ3>,
 <Atom CA>,
 <Atom C>,
 <Atom O>,
 <Atom CB>,
 <Atom CG2>,
 <Atom OG1>,
 <Atom CD2>,
 <Atom CE1>,
 <Atom CG>,
 <Atom NE2>,
 <Atom CE1>,
 <Atom CD1>,
 <Atom OXT>,
 <Atom CB>,
 <Atom CD1>,
 <Atom CD2>,
 <Atom CE1>,
 <Atom CE2>,
 <Atom CG>,
 <Atom CZ>,
 <Atom OH>,
 <Atom CD>,
 <Atom CG>,
 <Atom NE2>,
 <Atom OE1>,
 <Atom N>,
 <Atom CA>,
 <Atom C>,
 <Atom CB>,
 <Atom CG1>,
 <Atom CG2>,
 <Atom N>,
 <Atom C>,
 <Atom O>,
 <Atom CH2>,
 <Atom CZ2>,
 <Atom CZ3>,
 <Atom CA>,
 <Atom C>,
 <Atom O>,
 <Atom CB>,
 <Atom CG2>,
 <Atom OG1>,
 <Atom CD2>,
 <Atom CG>,
 <Atom NE2>,
 <Atom CE1>,
 <Atom CZ>,
 <Atom CB>,
 <Atom CD1>,
 <Atom CG>,
 <Atom C>,
 <Atom O>,
 <Atom OXT>,
 <Atom

In [212]:
a[:,0] #res 1 #this number is 1 less 
d0 = [a[:,0][1], a[:,0][2],  a[:,0][3]]

#r2 = a[:,1] #res 2
d1 = [a[:,0][1],  a[:,1][2],  a[:,1][3]]

d2 = [a[:,0][2],  a[:,1][2], a[:,2][3]] # for the third residue, it took the 2nd of the 1st res's dostamce, as its first, and the second of the second residue, as its second

d3 = [a[:,0][3], a[:,1][3], a[:,2][3]] #takes last of first residue as first, last of second as second, last of third as third
#a[:,k] where k is 1 less than the len of residues
#
n_res = 4
for i in range(3, n_res - 1):
    print(i)
    d0 = [1,2,3]
    d1 = [d0[i-2], 2, 3] #[square 0], new, new [d0[1]]
    d2 = [d0[i-1], d1[i-1], 3]#d0[2], d1[2] new
    d3 = [d0[i], d1[i], [d2[i]]] d0[3] d1[3] d2[3]

d0,d1,d2,d3

([91.43030548095703, 102.7194595336914, 115.15265655517578],
 [91.43030548095703, 116.49356079101562, 103.4726791381836],
 [102.7194595336914, 116.49356079101562, 30.47676658630371],
 [115.15265655517578, 103.4726791381836, 30.47676658630371])

In [217]:
ds=[]
for i in range(n_res - 1):
    #for j in range(1, n_res):
    #print(i)
    d = [a[:,i][j] for j in range(1, n_res)]
    d_next = [[[], []] +]
    counter += 1

    ds.append(d)
    print(ds)




[[91.43030548095703, 102.7194595336914, 115.15265655517578]]
[[91.43030548095703, 102.7194595336914, 115.15265655517578], [0.0, 116.49356079101562, 103.4726791381836]]
[[91.43030548095703, 102.7194595336914, 115.15265655517578], [0.0, 116.49356079101562, 103.4726791381836], [116.49356079101562, 0.0, 30.47676658630371]]


data types

In [ ]:
a[:,0] #res 1 #this number is 1 less 
d01, d02, d03 = a[:,0][1], a[:,0][2],  a[:,0][3]

r2 = a[:,1] #res 2
d10, d12, d13 =  a[:,0][1],  a[:,1][2],  a[:,1][3]

d20, d21, d23 = a[:,0][2],  a[:,1][2], a[:,2][3] # for the third residue, it took the 2nd of the 1st res's dostamce, as its first, and the second of the second residue, as its second

d30,d31,d32 = a[:,0][3], a[:,1][3], a[:,2][3] #takes last of first residue as first, last of second as second, last of third as third
#a[:,k] where k is 1 less than the len of residues
#
n_res = 4
for i in range(3, n_res):
    print(i)
    d30, d31, d32 = [res 0 distances][i], [res 1 d's][index of residue], []



In [180]:
for residue in cutouts_apdb[0]:
    print(residue)

[(0, array([20.947, 40.838, 23.05 ], dtype=float32), 'T', [<Atom OXT>, <Atom NE2>, <Atom CD>, <Atom OE1>, <Atom O>, <Atom CG>, <Atom CB>, <Atom NE2>, <Atom CD2>, <Atom CE1>, <Atom CG>, <Atom CD1>, <Atom O>, <Atom OG1>, <Atom CZ2>, <Atom CA>, <Atom CB>, <Atom CG2>, <Atom C>, <Atom OH>, <Atom O>, <Atom CZ>, <Atom CB>, <Atom CG>, <Atom CD2>, <Atom CE2>, <Atom CG2>, <Atom CE1>, <Atom CZ>, <Atom C>, <Atom N>, <Atom C>, <Atom CA>, <Atom N>, <Atom CH2>, <Atom CZ3>, <Atom O>, <Atom CG1>, <Atom CB>, <Atom CD1>, <Atom CG>, <Atom CB>])]
[(1, array([43.749, 21.409, 17.445], dtype=float32), 'T', [<Atom CE2>, <Atom OG1>, <Atom CA>, <Atom CB>, <Atom CD1>, <Atom CZ3>, <Atom C>, <Atom CA>, <Atom CH2>, <Atom O>, <Atom CG1>, <Atom CZ2>, <Atom N>, <Atom CB>, <Atom C>, <Atom CG2>, <Atom CA>, <Atom OE1>, <Atom C>, <Atom CE1>, <Atom O>, <Atom CG2>, <Atom CD2>, <Atom C>, <Atom O>, <Atom OXT>, <Atom OH>, <Atom CE1>, <Atom CZ>, <Atom CD1>, <Atom CE2>, <Atom CG>, <Atom CD2>, <Atom NE2>, <Atom CD>, <Atom CG>, <At

In [269]:
distance_cutoff=8
#rcut = 8, rcrit = 7
ns = PDB.NeighborSearch(list(structure.get_atoms()))
for atom in residue.get_atoms():
    atomN = str(atom)[6:-1]
    if atomN in ["OE2", "SG", "NZ", "OD2", "OH", "NE2", "ND1"]:
        center = atom
        cutout = ns.search(atom.get_coord(), distance_cutoff, "A")
    #a=[str(atom)[5:-1] for atom in cutout if str(atom)[5:-1] in ["OE2", "SG", "NZ", "OD2", "OH", "NE2", "ND1"]]
print(center)
[(atom.get_coord(), atom.get_parent().get_resname(), atom) for atom in cutout if atom != center and str(atom)[6:-1] in ["OE2", "SG", "NZ", "OD2", "OH", "NE2", "ND1"]]

<Atom OE2>


[(array([36.05 , 44.187, 18.722], dtype=float32), 'GLN', <Atom NE2>),
 (array([44.3  , 52.054, 19.498], dtype=float32), 'ASP', <Atom OD2>)]

In [243]:
[atom.get_coord() for atom in cutout if str(atom)[6:-1] in ["OE2", "SG", "NZ", "OD2", "OH", "NE2", "ND1"]]

[array([43.32 , 47.36 , 18.973], dtype=float32)]

In [231]:
#for atom in cutout:
[atom.get_coord() for atom in cutout if str(atom)[6:-1] in ["OE2", "SG", "NZ", "OD2", "OH", "NE2", "ND1"]]
#print(str(atom)[5:-1])

[array([43.32 , 47.36 , 18.973], dtype=float32)]

In [ ]:
for atom in structure_list:
    

In [185]:
for atom in structure_list:
    for atom in structure.get_atoms():
        print(atom)

    except:
        pass

AttributeError: 'list' object has no attribute 'get_atoms'

In [169]:
structure_list

[]

In [1]:
#1 generate cutouts

#find the protonatable sites
pdb= '0.pdb' #ND2 = HD2, #NE2 = HIE, ND1=HID

In [ ]:
HisNe,HisNd, LysNZ, CysS, G,A,T=[],[], [],[],[],[],[] #NE2 unique to HIS
with open(pdb) as file:
    lines = file.readlines()
for line in lines:
    #print(line)
    try:
        Line=line.split()
        print(Line)
        #print(Line[2])
        if Line[2] == 'NE2': #his
            #print(2)
            HisNe.append(Line[3], Line[2],float(Line[5]),float(Line[6]),float(Line[7]))
            print(33)
        if Line[2] == 'ND1':
            HisNd.append(Line[3], Line[2],float(Line[5]),float(Line[6]),float(Line[7]))
            print(1)
        if Line[2]=="NZ": #LYS, delete HZ1
            LysNZ.append((Line[3],Line[2],float(Line[5]),float(Line[6]),float(Line[7]))) #coords
            print(2)
        if Line[2]=="SG": #cys
            CysS.append(((Line[3],Line[2],float(Line[5]),float(Line[6]),float(Line[7]))))
        if Line[2]== "OE2":#glu 
            G.append((Line[3],Line[2],float(Line[5]),float(Line[6]),float(Line[7])))
            print(3)
        if Line[2]== "OE2":#as[] #od1 or od2
            A.append((Line[3],Line[2],float(Line[5]),float(Line[6]),float(Line[7])))
            print(4)
        if Line[2] == "OH":#tyr
            T.append((Line[3],Line[2],float(Line[5]),float(Line[6]),float(Line[7])))
            print(5)
    except:
        pass
print(HisNe,HisNd,LysNZ, G,A,T)

centers = HisNe + HisNd + LysNZ + CysS + G + A + T
atom_coordinates = [(center[2:5]) for center in centers]
print(atom_coordinates)
ds=[]
r_crit=10
r_cutoff=11 #regular
import numpy as np
for i in range(len(atom_coordinates)):
    for j in range(i + 1, len(atom_coordinates)):
        print(coord1)
        coord1 = np.array(atom_coordinates[i])  # Exclude atom name
        coord2 = np.array(atom_coordinates[j][1:])  # Exclude atom name
        d=np.linalg.norm(coord2 - coord1)
        ds.append(d)
ds

In [151]:
HisNe,HisNd, LysNZ, CysS, G,A,T=[],[], [],[],[],[],[] #NE2 unique to HIS
with open(pdb) as file:
    lines = file.readlines()
for line in lines:
    #print(line)
    
    Line=line.split()
    if len(Line) > 7:
        #print(Line)
        if Line[2] == 'NE2': #his
            print((Line[3], Line[2],float(Line[5]),float(Line[6]),float(Line[7])))
            print(3)
            HisNe.append((Line[3],float(Line[5]),float(Line[6]),float(Line[7])))
            print(33)
        if Line[2] == 'ND1':
            print(Line[3], Line[2],float(Line[5]),float(Line[6]),float(Line[7]))
            HisNd.append((Line[3], float(Line[5]),float(Line[6]),float(Line[7])))
            print(1)
        if Line[2]=="NZ": #LYS, delete HZ1
            print(Line[3], Line[2],float(Line[5]),float(Line[6]),float(Line[7]))
            LysNZ.append((Line[3],float(Line[5]),float(Line[6]),float(Line[7]))) #coords
            print(2)
        if Line[2]=="SG": #cys
            CysS.append((Line[3],float(Line[5]),float(Line[6]),float(Line[7])))
        if Line[2]== "OE1":#glu #oe1 or oe2
            G.append((Line[3],float(Line[5]),float(Line[6]),float(Line[7])))
            print(3)
        if Line[2]== "OE2":#as[] #od1 or od2
            A.append((Line[3],float(Line[5]),float(Line[6]),float(Line[7])))
            print(4)
        if Line[2] == "OH":#tyr
            T.append((Line[3],float(Line[5]),float(Line[6]),float(Line[7])))
            print(5)
print(HisNe,HisNd,LysNZ, G,A,T)

centers = HisNe + HisNd + LysNZ + CysS + G + A + T
atom_coordinates = [center for center in centers]

ds=[]
r_c, r_cut = 11,12
import numpy as np
for i in range(len(atom_coordinates)):
    for j in range(i + 1, len(atom_coordinates)):
        ic, jc = atom_coordinates[i], atom_coordinates[j]
        print(ic)
        if ic[0] != jc[0]:
            coord1 = np.array(ic[1:])  # Exclude atom name
            coord2 = np.array(jc[1:]) # Exclude atom name
            print(coord1)
            d=np.linalg.norm(coord2 - coord1)
            #if d < rc:
                #rcs.append(d)
            
            ds.append(("".join([str(i),str(j)]),(d,coord1,coord2)))

 #min([rcs.append(d) if d < rc])

for i in [2]: #max index 2
    for string in [d[0] for d in ds]:
        if i in string:

for j in range(len(atom_coordinates)):
    



HID ND1 49.695 61.905 10.22
1
('HID', 'NE2', 48.021, 60.575, 10.332)
3
33
LYS NZ 38.859 65.19 11.863
2
[('HID', 48.021, 60.575, 10.332)] [('HID', 49.695, 61.905, 10.22)] [('LYS', 38.859, 65.19, 11.863)] [] [] []
('HID', 48.021, 60.575, 10.332)
('HID', 48.021, 60.575, 10.332)
[48.021 60.575 10.332]
('HID', 49.695, 61.905, 10.22)
[49.695 61.905 10.22 ]


['02', '12']

In [159]:
#def get_cutout(dask_df, distance_cutoff): #"PARENT" FUNCTION
from Bio.PDB import *
from Bio import PDB
from torch import nn
import numpy as np
from ase import Atoms, Atom
import dask.dataframe as dd
from ordered_set import OrderedSet
import os

pdbname="1a2p"  
pdb_parser = PDB.PDBParser()
pdbs = list(OrderedSet(list(dask_df["PDB ID"])))

for i in range(3,4): #will equal len of set of pdbs in pypka, == 121294 
    #get target protein information, check the protein structure for questionable atoms, and gets biopython structure object in their absense. 
    structure_list=[] #per pdb
    pdbname = pdbs[i]
    Structure = pdb_parser.get_structure("",  PDBList().retrieve_pdb_file(str.lower(pdbname),obsolete=False, pdir='PDB',file_format = 'pdb'))
    #Structure=pdb_parser.get_structure("", 'pdb11as.ent')
    structure, pdb_titratable_residues = check_atoms_protein(Structure, Structure.get_atoms())
    if not structure: #skip entire pdb and all its entries in pypka db if there are undesirables in pdb
        continue

    ns = PDB.NeighborSearch(list(structure.get_atoms())) #set u
    pdb_df = dask_df[dask_df.iloc[:, 1] == pdbname].drop(columns = ["PDB ID", "pKa"]) #make a subdf containing only residue entries which are in PYPKA (dask_df) TO Save time.
    #for each represented titratable residue in PYPKA, generate a cutout and saves to pdb 
    skipped_residues=[]
    for j in range(len(pdb_df)):  #go through each residue in a pdb
        chain=pdb_df.iloc[j]['Chain'] 
        res_id = int(pdb_df.iloc[j]['Res ID'])
        try:
            residue=structure[0][chain][res_id]
            pypka_resname, PDBresname = pdb_df.iloc[j]['Res Name'], residue.get_resname()
            if pypka_resname not in ['NTR', 'CTR', PDBresname]:
                continue
            #center = 
            cutout = ns.search(residue.center_of_mass(geometric=True), distance_cutoff, "A")
            #structure_list.cuts.append((counter, cut))out)
            atoms_to_structure(cutout, f"{pdbname}_{chain}_{res_id}_{pypka_resname}") #save as pdb
        except:
            f"pypka residue not found in pdb {pdbname}, skipping {res_id}, {chain})"

            pass
            #os.remove(f"{local_folder}/PDB/pdb{pdbname}.ent")
    lines = str(pdb_titratable_residues).strip("{}()'")
    with open(f"{pdbname}", 'w') as file: #save all the residue names 
        
        file.write(lines)
    #os.remove(f"{local_folder}/PDB/pdb{pdbname}.ent")

    

Structure exists: 'PDB/pdb11as.ent' 


/Users/jessihoernschemeyer/miniconda3/envs/cfcnn/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 5658.
  warnings.warn(
/Users/jessihoernschemeyer/miniconda3/envs/cfcnn/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 5667.
  warnings.warn(
/Users/jessihoernschemeyer/miniconda3/envs/cfcnn/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 5676.
  warnings.warn(
/Users/jessihoernschemeyer/miniconda3/envs/cfcnn/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 5720.
  warnings.warn(


NameError: name 'res_noarg' is not defined

In [ ]:
#cutout generation

cutout = ns.search(site_coords, r_cutoff, "A")

In [ ]:
"""if distance < min_distance:
    min_distance = distance
    min_pair = (atom_coordinates[i][0], atom_coordinates[j][0])
d=np.linalg.norm(res.center_of_mass(geometric=True) - atom.get_coord()) """

In [200]:
!cat 11as_A_245_ASP.pdb

ATOM      1  CD  ARG A 187       4.862  14.863   9.725  1.00 22.10           C  
ATOM      2  NE  ARG A 187       3.452  14.957  10.111  1.00 37.02           N  
ATOM      3  CG  ARG A 187       5.508  13.548  10.121  1.00 20.65           C  
ATOM      4  CB  ARG A 187       6.942  13.798  10.539  1.00  5.41           C  
ATOM      5  CZ  ARG A 187       2.946  15.896  10.911  1.00 28.14           C  
ATOM      6  CA  ARG A 187       7.716  12.600  11.096  1.00 16.09           C  
ATOM      7  NH1 ARG A 187       3.674  16.962  11.219  1.00 23.52           N  
ATOM      8  OD2 ASP A 245       7.984  19.262  10.396  1.00  8.53           O  
ATOM      9  OD1 ASP A 245       6.936  17.457  11.043  1.00 18.81           O  
ATOM     10  CG  ASP A 245       7.454  18.567  11.287  1.00 15.48           C  
ATOM     11  CB  ASP A 245       7.454  19.087  12.720  1.00 16.02           C  
ATOM     12  CA  ASP A 245       6.934  18.056  13.732  1.00  9.45           C  
ATOM     13  N   ASP A 245  

Structure exists: 'PDB/pdb11as.ent' 


/Users/jessihoernschemeyer/miniconda3/envs/cfcnn/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 5658.
  warnings.warn(
/Users/jessihoernschemeyer/miniconda3/envs/cfcnn/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 5667.
  warnings.warn(
/Users/jessihoernschemeyer/miniconda3/envs/cfcnn/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 5676.
  warnings.warn(
/Users/jessihoernschemeyer/miniconda3/envs/cfcnn/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 5720.
  warnings.warn(
